___
# Lecture 3 - Regression and Interpretting Statistical Significance
___

## Review

* `PCA` is a good way to assess the quality of our data
* We can create informative plots of our data with `matplotlib`
* `Pandas` dataframes can be filtered/subsetted
* **Data pre-processing is important to remove noise for downstream analysis!**

## Lesson Outline

#### (1) [Introduction to regression](#1)
#### (2) [Motivation](#2)
#### (3) [Setup](#3)
#### (4) [Regression for feature selection](#4)
#### (5) [Additional Materials](#5)
   - [Extra challenges](#5a)
   - [Extra resources](#5b)

## Introduction to Regression <a class="anchor" id="1"></a>

#### Linear regression:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/1200px-Linear_regression.svg.png" width="500" height="500">

\begin{equation*}\LARGE Y(x_i) = B_0 + B_1x_i + \epsilon_i \end{equation*}

Note: `Y` is our `dependent` variable, and our `x` is our `independent` variable

## Motivation <a class="anchor" id="2"></a>

#### Can be used for `classification` and `feature selection`

#### Examples from research
* Genome Wide Association Studies (GWAS)
* Drug sensitivity
* CRISPR off-target effects

## Setup <a class="anchor" id="3"></a>

Import neccessary packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load expression data

In [ ]:
expr_data_meta = pd.read_csv("inSphero.abundance.table_edit190410_filtered.csv",index_col=0)
expr_data = expr_data_meta.drop("symbol",axis=1)
annots = expr_data_meta.loc[:,["symbol"]]

In [ ]:
expr_data.head()

<h3 style="color:red;"> What are our dependent and independent variables? </h3>

It may also be beneficial to use the description file

In [ ]:
description = pd.read_csv("inSphero_doe_edit190410.csv",index_col=1)
description.head()

**Problem:** The `baseline` names in `description` and `expr_data` do not match 

<h3 style="color:red;"> How can we fix this? </h3>

In [ ]:
description.head()

## Regression for Feature Selection <a class="anchor" id="4"></a>

### Data formatting

We want to identify genes that separate treatment and control. We will test each gene separately. First we need to create a phenotype dataframe of our `depedent` variable. 

In [ ]:
phenos = pd.DataFrame(index=expr_data.columns, columns=["group.function","day","outcome"])
phenos

<h3 style="color:red;"> Exercise: Fill out the phenos dataframe. </h3>

The `group.function` column can be copied from the `description` dataframe.

The `day` column can **also** be copied from the `description` dataframe, but lets make this an `integer` instead of `string`.

**Example:** 
```
day0 -> 0
```

**Hint:** Strings can be indexed or split! 
```python
s = "Hello World"
s_fix1 = s[:5] 
s_fix2 = s.split(" ")[0]
s_fix1 == s_fix2
```

Now we need an `outcome` column, which will be a **binary** variable where `0 = control` and `1 = drug`. 

We can fill this out programmatically using Python's `if`/`in` statement and the `group.function` **OR** by brute-force like I did to color our scatter plot in `Lesson 2`!

**Hint:**
```python
color = ["blue"]*3+...+["red"]*2
```

Using brute-force

Programmatically

Now the `phenos` dataframe should be complete!

In [ ]:
phenos

### Model setup

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot

In [ ]:
def plot_gene(gene,expr_data,phenos,model):
    """
    Function to plot gene expression
    """
    fig = plt.figure()
    ax = plt.axes()
    # Plot controls
    ax.scatter(expr_data.loc[gene,phenos.outcome==0],phenos[phenos.outcome==0].outcome,
           label="control",s=100)
    # Plot cases
    ax.scatter(expr_data.loc[gene,phenos.outcome==1],phenos[phenos.outcome==1].outcome,
           label="treatment",s=100)
    # Plot regression line
    abline_plot(model_results=model, ax=ax, color="black",linestyle="--")
    ax.set_xlabel("Gene Expression ln(tpm)",fontsize=14)
    ax.set_ylabel("Treatment Status",fontsize=14)
    ax.set_title("Gene: {}".format(gene),fontsize=14)
    ax.legend(fontsize=12,bbox_to_anchor=(1.4,1))
    plt.show()

### Apply to data

First lets use `linear regression` to fit our the gene `ENSG00000255689` in our data.

What is our `X` and what is our `y`?

In [ ]:
X = 
y = 

In [ ]:
ols = sm.OLS(y,X).fit()

In [ ]:
ols.summary()

In [ ]:
plot_gene('ENSG00000255689',expr_data,phenos,ols)

<h3 style="color:red;"> What are we missing? </h3>

\begin{equation*}\large Y(x_i) = B_0 + B_1x_i + \epsilon_i \end{equation*}

<h3 style="color:red;"> Exercise: Run the linear model for ENSG00000137563. What is different about this example?</h3>

In [ ]:
X = 
y = 

ols = 

plot_gene('ENSG00000137563',expr_data,phenos,ols)

`Generalized linear models`(GLMs) are a flexible version of the linear regression model. While linear regression assumes the `dependent` variable is **normally distributed**, we can use a `GLM` to apply these same concepts to our **binary** dataset. For example `logistic regression` is a type of `GLM` that uses log transformation: $logit(Y(x_i))$.

In [ ]:
glm = sm.GLM(y,X,family=sm.families.Binomial()).fit(maxiter=1)

In [ ]:
plot_gene('ENSG00000137563',expr_data,phenos,glm)

In [ ]:
ols.predict(np.array([1,4.6]))

In [ ]:
glm.predict(np.array([1,4.6]))

<h3 style="color:red;"> Exercise: Perform regression for all genes and save coeficients and p-values to a dataframe. </h3>

There are a couple errors here, find them and fix them. **Use the error messages to guide you to the lines that need fixed!**

**Hint:** When debugging code un a `for` loop it helps to look at just one element at a time. You can either do this with `print` statements or by subsetting data to one or a few examples `expr_data.index[:n]`.

In [ ]:
import tqdm

In [ ]:
coefs_df = pd.DataFrame(index=expr_data.index,columns=["coef","p-value"])
# this is a dictionary, we will use it to hold our models so we can make plots later
models = {}
# Loop through genes
for gene in tqdm.tqdm(expr_data.index):
    X = expr_data.loc[:,gene].values
    y = phenos.loc["outcome",:].values
    # Fit model
    lr = sm.GLM(X,y,family=sm.families.Binomial()).fit(maxiter=1)
    models[gene] = lr
    # Extract coeficient and p-value for the gene
    pvals = lr.pvalues
    coefs = lr.params
    coefs_df.loc[gene,:] = [coefs,pvals]

In [ ]:
coefs_df = coefs_df.sort_values(by="p-value")

In [ ]:
coefs_df.head()

### Interpret results

In [ ]:
plot_gene(coefs_df.index[0],expr_data,phenos,models[coefs_df.index[0]])
plt.show()

In [ ]:
plot_gene(coefs_df.index[-1],expr_data,phenos,models[coefs_df.index[-1]])
plt.show()

<h3 style="color:red;"> Exercise: Find the gene with the lowest p-value that is up-regulated in treatment cases. </h3>

## Additional Materials <a class="anchor" id="5"></a>

### Extra Challenges <a class="anchor" id="5a"></a>

#### Correct for multiple testing with permutation tests

In [ ]:
def perm_pval(X,y,true_coef,nperms=5000):
    """
    Perform permutation testing by shuffling the phenotypes
    
    Parameters
    ----------
    X : np.array
        Data matrix with features
    y : np.array
        Sample phenotypes
    true_coef : int , optional
        Number of times to permute your data
    
    Returns
    -------
    np.array
        p-values from permutation test 
    """
    coef_rand = []
    for i in range(nperms):
        y_rand = np.random.permutation(y)
        lr_rand = sm.GLM(y_rand,X,family=sm.families.Binomial())
        lr_rand = lr_rand.fit(maxiter=1)
        coef_rand.append(lr_rand.params[1])
    pval = sum(np.absolute(coef_rand)>=np.absolute(true_coef))/nperms
    return pval

#### Determine which genes are not significant given the permutation p-value

In [ ]:
coefs_df_mult = pd.DataFrame(index=expr_data.index,columns=["coef","p-value","perm_p"])
for gene in tqdm.tqdm(expr_data.index[:5]):
    X = sm.add_constant(expr_data.loc[gene,:].values)
    y = phenos.loc[:,"outcome"].values

    lr = sm.GLM(y,X,family=sm.families.Binomial()).fit(maxiter=1)

    pvals = lr.pvalues
    coefs = lr.params
    
    permP = perm_pval(X,y,coefs[1])
    coefs_df_mult.loc[gene,:] = [coefs[1],pvals[1],permP]

In [ ]:
coefs_df_mult.head()

### Extra Resources <a class="anchor" id="5b"></a>

#### Additional flavors of regression:
- `Regularization` 
    * A way to discourage model complexity ([bias-variance tradeoff](https://cdn-images-1.medium.com/max/1600/1*9hPX9pAO3jqLrzt0IE3JzA.png))
    * Used to avoid over-fitting to training data 
    * [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) 
, [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
, [Elastic Net](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)

- `Cox models`
    * Used to measure time-dependent effects
    * [Python tutorial](https://www.statsmodels.org/stable/duration.html)

- `Cross validation`
    * This is something we do to ensure we have the best model parameters without overfitting to our data.
    * [Further description and implementation](https://scikit-learn.org/stable/modules/cross_validation.html)